In [1]:
'''
Objective:
SVD for image compression example
Author:
Goran Trlin

Prerequisites:
pip install numpy
pip install pillow

Video tutorial URL:
https://www.youtube.com/watch?v=SU851ljMIZ8

'''

import numpy
from PIL import Image
import math

# FUNCTION DEFINTIONS:

# open the image and return 3 matrices, each corresponding to one channel (R, G and B channels)
def openImage(imagePath):
    imOrig = Image.open(imagePath)
    im = numpy.array(imOrig)

    aRed = im[:, :, 0]
    aGreen = im[:, :, 1]
    aBlue = im[:, :, 2]

    return [aRed, aGreen, aBlue, imOrig]


# compress the matrix of a single channel
def compressSingleChannel(channelDataMatrix, singularValuesLimit):
    uChannel, sChannel, vhChannel = numpy.linalg.svd(channelDataMatrix)
    aChannelCompressed = numpy.zeros((channelDataMatrix.shape[0], channelDataMatrix.shape[1]))
    k = singularValuesLimit

    leftSide = numpy.matmul(uChannel[:, 0:k], numpy.diag(sChannel)[0:k, 0:k])
    aChannelCompressedInner = numpy.matmul(leftSide, vhChannel[0:k, :])
    aChannelCompressed = aChannelCompressedInner.astype('uint8')
    return aChannelCompressed

# Peak signal-to-noise ratio (PSNR) is the ratio between the maximum possible power
# of an image and the power of corrupting noise that affects the quality of its representation. 
# PSNR is most commonly used to estimate the efficiency of compressors, filters, etc. The larger 
# the value of PSNR, the more efficient is a corresponding compression or filter method.
def PSNR(original, compressed):
    mse = numpy.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * math.log10(max_pixel / math.sqrt(mse))
    return psnr

# MAIN PROGRAM:
# print('*** Image Compression using SVD \n')
aRed, aGreen, aBlue, originalImage = openImage('lena.png')
#print("orig image size = ",originalImage.size)
# image width and height:

imageWidth = originalImage.size[0]
imageHeight = originalImage.size[1]

# selected k values
singularValuesLimit = 64

aRedCompressed = compressSingleChannel(aRed, singularValuesLimit)
aGreenCompressed = compressSingleChannel(aGreen, singularValuesLimit)
aBlueCompressed = compressSingleChannel(aBlue, singularValuesLimit)

imr = Image.fromarray(aRedCompressed, mode=None)
img = Image.fromarray(aGreenCompressed, mode=None)
imb = Image.fromarray(aBlueCompressed, mode=None)

newImage = Image.merge("RGB", (imr, img, imb))

# originalImage.show()
# newImage.show()

# CALCULATE AND DISPLAY THE COMPRESSION RATIO
mr = imageHeight
mc = imageWidth

originalSize = mr * mc * 3
compressedSize = singularValuesLimit * (1 + mr + mc) * 3

# print('original size:')
# print(originalSize)

# print('compressed size:')
# print(compressedSize)

# print('Ratio compressed size / original size:')

im = Image.open('lena.png')
imOrig = numpy.array(im)
imComp = numpy.array(newImage)


# print("\nPeak signgal to noise ratio(PSNR) of SVD  "+str(singularValuesLimit)+" colors = "+str(round(PSNR(imOrig,imComp),2)))

AttributeError: 'PngImageFile' object has no attribute 'shape'

In [3]:
import numpy as np
from sewar import full_ref
from skimage import measure


ref_img = imOrig
img = imComp

ratio = compressedSize * 1.0 / originalSize
print("compression ratio(CR) of SVD " +str(singularValuesLimit)+" colors = "+str(int(ratio*100))+"%")

mse_skimg = measure.compare_mse(ref_img, img)
print("MSE: based on scikit-image = ", mse_skimg)

#Same as PSNR available in sewar
psnr_skimg = measure.compare_psnr(ref_img, img, data_range=None)
print("PSNR: based on scikit-image = ", psnr_skimg)

# #Normalized root mean squared error
# rmse_skimg = measure.compare_nrmse(ref_img, img)
# print("RMSE: based on scikit-image = ", rmse_skimg)


###############################################################
#ERGAS Global relative error
"""calculates global relative error 
GT: first (original) input image.
P: second (deformed) input image.
r: ratio of high resolution to low resolution (default=4).
ws: sliding window size (default = 8).
	:returns:  float -- ergas value.
	"""
ergas_img = full_ref.ergas(ref_img, img, r=4, ws=8)
print("EGRAS: global relative error = ", ergas_img)



####################################################################
#Multiscale structural similarity index
"""calculates multi-scale structural similarity index (ms-ssim).
	:param GT: first (original) input image.
	:param P: second (deformed) input image.
	:param weights: weights for each scale (default = [0.0448, 0.2856, 0.3001, 0.2363, 0.1333]).
	:param ws: sliding window size (default = 11).
	:param K1: First constant for SSIM (default = 0.01).
	:param K2: Second constant for SSIM (default = 0.03).
	:param MAX: Maximum value of datarange (if None, MAX is calculated using image dtype).
	:returns:  float -- ms-ssim value.
	"""
# msssim_img=full_ref.msssim(ref_img, img, weights=[0.0448, 0.2856, 0.3001, 0.2363, 0.1333], ws=11, K1=0.01, K2=0.03, MAX=None)

# print("MSSSIM: multi-scale structural similarity index = ", msssim_img)


##############################################################################
#PSNR
"""calculates peak signal-to-noise ratio (psnr).
	:param GT: first (original) input image.
	:param P: second (deformed) input image.
	:param MAX: maximum value of datarange (if None, MAX is calculated using image dtype).
	:returns:  float -- psnr value in dB.
	"""
# psnr_img=full_ref.psnr(ref_img, img, MAX=None)

# print("PSNR: peak signal-to-noise ratio = ", psnr_img)


##########################################################################
#PSNRB: Calculates PSNR with Blocking Effect Factor for a given pair of images (PSNR-B)
"""Calculates PSNR with Blocking Effect Factor for a given pair of images (PSNR-B)
	:param GT: first (original) input image in YCbCr format or Grayscale.
	:param P: second (corrected) input image in YCbCr format or Grayscale..
	:return: float -- psnr_b.
	"""
#psnrb_img = full_ref.psnrb(ref_img, img)

#print("PSNRB: peak signal-to-noise ratio with blocking effect = ", psnrb_img)

#######################################################################
#relative average spectral error (rase)
"""calculates relative average spectral error (rase).
	:param GT: first (original) input image.
	:param P: second (deformed) input image.
	:param ws: sliding window size (default = 8).
	:returns:  float -- rase value.
	"""
RASE_img = full_ref.rase(ref_img, img, ws=8)
#print("RASE: relative average spectral error = ", RASE_img)


######################################################################
#RMSE
"""calculates root mean squared error (rmse).
	:param GT: first (original) input image.
	:param P: second (deformed) input image.
	:returns:  float -- rmse value.
	"""
rmse_img = full_ref.rmse(ref_img, img)
print("RMSE: root mean squared error = ", rmse_img)



######################################################################
#root mean squared error (rmse) using sliding window
"""calculates root mean squared error (rmse) using sliding window.
	:param GT: first (original) input image.
	:param P: second (deformed) input image.
	:param ws: sliding window size (default = 8).
	:returns:  tuple -- rmse value,rmse map.	
	"""
rmse_sw_img = full_ref.rmse_sw(ref_img, img, ws=8)
#print("RMSE_SW: root mean squared error with sliding window = ", rmse_sw_img)


#########################################################################
#calculates spectral angle mapper (sam).
"""calculates spectral angle mapper (sam).
	:param GT: first (original) input image.
	:param P: second (deformed) input image.
	:returns:  float -- sam value.
	"""
ref_sam_img = full_ref.sam(ref_img, img)
print("REF_SAM: spectral angle mapper = ", ref_sam_img)


######################################################################
#Spatial correlation coefficient
full_ref.scc(ref_img, img, win=[[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]], ws=8)

#Structural similarity index
"""calculates structural similarity index (ssim).
	:param GT: first (original) input image.
	:param P: second (deformed) input image.
	:param ws: sliding window size (default = 8).
	:param K1: First constant for SSIM (default = 0.01).
	:param K2: Second constant for SSIM (default = 0.03).
	:param MAX: Maximum value of datarange (if None, MAX is calculated using image dtype).
	:returns:  tuple -- ssim value, cs value.
	"""
# ssim_img = full_ref.ssim(ref_img, img, ws=11, K1=0.01, K2=0.03, MAX=None, fltr_specs=None, mode='valid')
# print("SSIM: structural similarity index = ", ssim_img)

##############################################################################
#Universal image quality index
"""calculates universal image quality index (uqi).
	:param GT: first (original) input image.
	:param P: second (deformed) input image.
	:param ws: sliding window size (default = 8).
	:returns:  float -- uqi value.
	"""
UQI_img = full_ref.uqi(ref_img, img, ws=8)
print("UQI: universal image quality index = ", UQI_img)

##############################################################################
#Pixel Based Visual Information Fidelity (vif-p)
"""calculates Pixel Based Visual Information Fidelity (vif-p).
	:param GT: first (original) input image.
	:param P: second (deformed) input image.
	:param sigma_nsq: variance of the visual noise (default = 2)
	:returns:  float -- vif-p value.
	"""
VIFP_img = full_ref.vifp(ref_img, img, sigma_nsq=2)
print("VIFP: Pixel Based Visual Information Fidelity = ", VIFP_img)

compression ratio(CR) of SVD 64 colors = 25%
MSE: based on scikit-image =  77.4328104654948
PSNR: based on scikit-image =  29.2415533836655
EGRAS: global relative error =  2837.9029937416744
RMSE: root mean squared error = 

<ipython-input-3-7e501ff4caf6>:12: UserWarning: DEPRECATED: skimage.measure.compare_mse has been moved to skimage.metrics.mean_squared_error. It will be removed from skimage.measure in version 0.18.
  mse_skimg = measure.compare_mse(ref_img, img)
<ipython-input-3-7e501ff4caf6>:16: UserWarning: DEPRECATED: skimage.measure.compare_psnr has been moved to skimage.metrics.peak_signal_noise_ratio. It will be removed from skimage.measure in version 0.18.
  psnr_skimg = measure.compare_psnr(ref_img, img, data_range=None)


 8.79959149423965
REF_SAM: spectral angle mapper =  0.06716089694332256
UQI: universal image quality index =  0.993927870935864
VIFP: Pixel Based Visual Information Fidelity =  0.41409159651510524
